# **Extract DayMET Data**

By Bridget Bittmann

Date created: March 28, 2022

Date editted: May 4, 2022

Purpose: This script extracts DayMET data from Google Earth Engine using the Earth Engine API in Python. This script also copied over a SSEBop ET data dataset and calculated zonal stats based on polygons. 

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=iPF02UM7JDdZNYmejE1xqMllLEZZ9tzQ2PLSFRofwqY&tc=CKRXRongZwbvKNdrGuU1LfFfDbUpps8Vk_myyfiiZ0g&cc=MJ-nhsgVgOAH7xl1kOa1AsQs4qbv7cUmHU6pxATY69w

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi61x9tKeIg5s0hcdU8tGpmbUIgfjsiDHHv1N-q-HbGd9FZZVRxoYo

Successfully saved authorization token.


In [2]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.2 MB/s 
     |████████████████████████████████| 16.7 MB 332 kB/s 
     |████████████████████████████████| 6.3 MB 26.2 MB/s 


In [3]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Go to correct folder in Google Drive
%cd drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
climate_stats/         irrigation_companies/  lcmap_files/   subset_test_shp/
diversion_timeseries/  irrig_lbrb/            masked/
extra_figures/         IrrMapper/             output_files/
hydromet_data/         LBRB_shp/              POUs/


In [10]:
## ----------------------------------- ## 
## 1. Import shapefile to clip dataset ##
## ----------------------------------- ## 

shp_file = 'POUs/POUs_EDIT_060622_Merge.shp'
subset = emap.shp_to_ee(shp_file) # converts shapefile to feature in GEE

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(subset, 0, 2), {}, 'POU')
map.addLayerControl()
map

In [6]:
## --------------------------------------- ##
## 2. IMPORT THE DAYMET DATA FOR MAX TEMPS ##
## --------------------------------------- ##

years = np.arange(1987,2021)
mean_max = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection
  mxtmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(years[i])}) #select the bands to analyze
  mean_max.append(mxtmp) #calculate the mean across all pixels

means_max_temp = ee.ImageCollection(mean_max) #convert list of image to image collection for zonal stats command

maximumTemperatureVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(means_max_temp, maximumTemperatureVis, 'tmax')
Map

In [ ]:
new_df = spatial_reduction(image_collection=means_max_temp, 
                  shape_file = subset,
                  reducer = ee.Reducer.mean(),
                  scale=1000,
                  file_name = 'function_try',
                  format = 'CSV',
                  output_folder = 'climate_stats',
                  name = 'max_temp_mean')

{'state': 'READY', 'description': 'function_try', 'creation_timestamp_ms': 1653077709471, 'update_timestamp_ms': 1653077709471, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'ICAP2LDAVQ3GJGL3ZZQXFQL6', 'name': 'projects/earthengine-legacy/operations/ICAP2LDAVQ3GJGL3ZZQXFQL6'}
{'state': 'RUNNING', 'description': 'function_try', 'creation_timestamp_ms': 1653077709471, 'update_timestamp_ms': 1653077717786, 'start_timestamp_ms': 1653077717750, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': 'ICAP2LDAVQ3GJGL3ZZQXFQL6', 'name': 'projects/earthengine-legacy/operations/ICAP2LDAVQ3GJGL3ZZQXFQL6'}


In [7]:
## ------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION ##
## ------------------------------------------- ##

years = np.arange(1987,2021)
sum_pr = []
for i in years:
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-10-01'), (str((i+1))+'-9-30')) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index':str(i)}) #select the bands to analyze
  sum_pr.append(prcp) #calculate the mean across all pixels

sum_precip = ee.ImageCollection(sum_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(sum_precip, precip_vis, 'prcp')
Map

In [8]:
## --------------------------------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION THROUGH IRRIGATION SEASON ##
## --------------------------------------------------------------------- ##

years = np.arange(1987,2021)
irrig_pr = []
for i in years:
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-03-15'), (str((i+1))+'-10-15')) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index':str(i)}) #select the bands to analyze
  irrig_pr.append(prcp) #calculate the mean across all pixels

irrig_precip = ee.ImageCollection(irrig_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(sum_precip, precip_vis, 'prcp')
Map

In [9]:
## ---------------------- ## 
## IMPORT MONTHLY ET DATA ##
## ---------------------- ## 

et_irrig = []

for y in years:
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-03-01'), str(y)+'-10-31')
  et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(y)}) # sum and convert to meters 
  et_irrig.append(et)

et_irrig = ee.ImageCollection(et_irrig)

et_cut = []

for y in years:
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-04-01'), str(y)+'-9-30')
  et_march = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-03-01'), str(y)+'-03-31')
  et_march = et_march.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': (str(y)+'-03')})
  et_oct = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-10-01'), str(y)+'-10-31')
  et_oct = et_oct.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': (str(y)+'-10')})
  et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(y)}) # sum and convert to meters 
  et_cut.append(et)
  et_cut.append(et_march)
  et_cut.append(et_oct)

et_cut = ee.ImageCollection(et_cut)
  

In [11]:
## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('climate_stats/maxtemp_stats.csv')
emap.zonal_statistics(means_max_temp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/precip_stats.csv')
emap.zonal_statistics(sum_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/irrig_precip_stats.csv')
emap.zonal_statistics(irrig_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/et.csv')
emap.zonal_statistics(et_irrig, subset, out_stats, statistics_type='MEAN', scale=30)

out_stats = os.path.join('climate_stats/et_cut.csv')
emap.zonal_statistics(et_cut, subset, out_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/maxtemp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/precip_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/irrig_precip_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/et.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/et_cut.csv


In [14]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

years = np.arange(1987,2021)
precip = pd.read_csv('climate_stats/precip_stats.csv')
max_temp = pd.read_csv('climate_stats/maxtemp_stats.csv')
irrig_pre = pd.read_csv('climate_stats/irrig_precip_stats.csv')
et_irrig = pd.read_csv('climate_stats/et.csv')
et_cut_ir = pd.read_csv('climate_stats/et_cut.csv')

names = et_cut_ir['WaterRight']

cut = pd.DataFrame(et_cut_ir['WaterRight'])
for i in years:
  fill = str(i)
  cut[fill] = et_cut_ir[et_cut_ir.columns[et_cut_ir.columns == (str(i)+'_et')]]


for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['Precip_mm'] = precip.iloc[i,0:34].values
  df['Max_temp'] = max_temp.iloc[i,0:34].values
  df['et_whole'] = et_irrig.iloc[i,0:34].values
  df['et_cut'] = cut.iloc[i,1:35].values
  out_path = os.path.join('climate_stats/final/'+names[i]+'_climate.csv')
  df.to_csv(out_path)


,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Warm Springs,224.624849,30.39189,1.103298,0.997727
1,1988,Warm Springs,322.606164,32.340458,1.068796,1.00729
2,1989,Warm Springs,288.785602,31.445234,1.154084,1.042377
3,1990,Warm Springs,234.922849,31.334732,1.097761,0.978567
4,1991,Warm Springs,232.945261,31.07256,1.081211,1.020322
5,1992,Warm Springs,437.746837,30.968933,1.132115,1.043714
6,1993,Warm Springs,199.709617,26.524097,0.791732,0.728642
7,1994,Warm Springs,419.19504,32.455408,1.05504,0.94972
8,1995,Warm Springs,410.091795,29.514194,1.148201,0.995529
9,1996,Warm Springs,453.629767,31.724796,1.170578,1.0


In [ ]:
## --------------- ## 
## COMPARE ET DATA ##
## --------------- ## 

years = np.arange(1987,2021)
et_whole = pd.read_csv('climate_stats/et.csv').drop(columns = ['Shape_Area', 'system:index', 'Shape_Leng'], axis=1)
et_cut = pd.read_csv('climate_stats/et_cut.csv')


names = et_whole['WaterRight']
et = []

march = pd.DataFrame(et_cut['WaterRight'])
october = pd.DataFrame(et_cut['WaterRight'])
cut = pd.DataFrame(et_cut['WaterRight'])

for i in years:
  fill = str(i)
  march[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'-03_et')]]
  october[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'-10_et')]]
  cut[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'_et')]]

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['et_whole'] = et_whole.iloc[i,0:34].values
  df['et_cut'] = cut.iloc[i,1:35].values
  df['march'] = march.iloc[i,1:35].values
  df['october'] = october.iloc[i,1:35].values
  et.append(df)

et = pd.concat(et)

et['diff'] = et['et_whole'] - et['et_cut']

avg = et.groupby('DIV_NAME')['et_whole', 'et_cut', 'diff', 'march', 'october'].mean()
avg.to_csv('climate_stats/et_avgs.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
